# Import all needed Python Libraries

In [2]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import ipyfilechooser
from openpyxl import load_workbook
import re
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import time
from tqdm.notebook import tqdm
import ipywidgets as widgets
from IPython.display import display, clear_output

## Choosing Excel file

In [3]:
# File chooser
chooser = ipyfilechooser.FileChooser()
display(chooser)

FileChooser(path='C:\Users\User\PycharmProjects\geoai', filename='', title='', show_hidden=False, select_desc=…

<font color='red'>Il file deve essere selezionato tramite il bottone superiore altrimenti l'analisi non può essere iniziata</font>

In [4]:
# Load the Excel file select only some columns
df_full = pd.read_excel(chooser.value, engine="openpyxl") #, usecols=[0,11, 12,13,17,18,22,23])  # Use engine="openpyxl" for .xlsx files

In [5]:
# Display the first few rows
df_full

,id,indirizzo,civico,cap,citta,provincia,regione,nazione
0,1,via luigi olivares,59,NaN,roma,RM,lazio,NaN
1,2,via luigi olivares,79,NaN,roma,RM,lazio,NaN
2,3,via francesco sapori,45,NaN,roma,RM,lazio,NaN


In [6]:
# Create a multi-select dropdown for column selection
column_selector = widgets.SelectMultiple(
    options=df_full.columns,
    description="Columns:",
    layout=widgets.Layout(width='50%'),
    style={'description_width': 'initial'}
)

In [7]:
# Button to confirm selection
confirm_button = widgets.Button(
    description="Load Selected Columns",
    button_style='success'
)

In [8]:
# Output widget to display DataFrame
output = widgets.Output()
output

Output()

In [20]:
# Function to update the output based on selection
def load_selected_columns(b):
    with output:
        global selected_columns, df_selected
        output.clear_output()  # Clear previous outputs
        selected_columns = list(column_selector.value)
        if selected_columns:
            df_selected = df_full[selected_columns]
            display(df_selected.head())  # Show first 5 rows of the new dataframe
        else:
            print("No columns selected!")

In [21]:
# Link button click event to function
confirm_button.on_click(load_selected_columns)

# Display widgets
display(column_selector, confirm_button, output)

SelectMultiple(description='Columns:', index=(1, 2, 4), layout=Layout(width='50%'), options=('id', 'indirizzo'…

Button(button_style='success', description='Load Selected Columns', style=ButtonStyle())

Output()

In [23]:
selected_columns

['indirizzo', 'civico', 'citta']

In [32]:
df_selected["Address"] = df_selected[selected_columns[0]].astype(str) + ", " + df_selected[selected_columns[1]].astype(str) + ", " + df_selected[selected_columns[2]].astype(str)

C:\Users\User\AppData\Local\Temp\ipykernel_4048\1736290632.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_selected["Address"] = df_selected[selected_columns[0]].astype(str) + ", " + df_selected[selected_columns[1]].astype(str) + ", " + df_selected[selected_columns[2]].astype(str)


## Processing Address Columns

In [33]:
# Combine address components into a single string
df_selected

,indirizzo,civico,citta,Address
0,via luigi olivares,59,roma,"via luigi olivares, 59, roma"
1,via luigi olivares,79,roma,"via luigi olivares, 79, roma"
2,via francesco sapori,45,roma,"via francesco sapori, 45, roma"


# Geocoding

In [19]:
# Initialize Nominatim geocoder if faisl change the string of the user_agent
geolocator = Nominatim(user_agent="geo_app111")

## With Quality Indicators 

In [21]:
def get_location_with_quality_indicators(address):
    try:
        # Get the location
        location = geolocator.geocode(address, exactly_one=False, addressdetails=True)
        
        if not location:
            return "No results found"
        
        results = []
        for loc in location:
            quality_indicators = {
                # The raw data contains useful information about the match
                "raw": loc.raw,
                
                # 'importance' field - higher values suggest better matches
                "importance": loc.raw.get("importance", None),
                
                # Type of result (building, road, city, etc)
                "class": loc.raw.get("class", None),
                "type": loc.raw.get("type", None),
                
                # How specific the result is (address details)
                "address_detail_count": len(loc.raw.get("address", {})),
                
                # If coordinates exist and are precise
                "has_coordinates": bool(loc.latitude and loc.longitude),
                
                # Display name (completeness can indicate quality)
                "display_name": loc.address
            }
            
            results.append({
                "location": loc,
                "quality": quality_indicators
            })
        
        return results
    
    except GeocoderTimedOut:
        return "Geocoding service timed out"
    except Exception as e:
        return f"Error: {str(e)}"

In [31]:
# Usage example
# address = "VIA LUDOVICO ARIOSTO, 32, FISCAGLIA, FE, FERRARA"
# address = "SOTTOPASSO DELLA COSTITUZIONE, 80143, NAPOLI"
address = "VIA DEL GIGNORO 19,50135,FI,FIRENZE"

results = get_location_with_quality_indicators(address)

In [32]:
# Analyze the results with proper error handling
try:
    if isinstance(results, list) and results:
        first_result = results[0]
        print(f"Found: {first_result['location'].address}")
        print(f"Importance score: {first_result['quality']['importance']}")
        print(f"Type: {first_result['quality']['class']}/{first_result['quality']['type']}")
        print(f"Address detail count: {first_result['quality']['address_detail_count']}")
    else:
        print(f"No valid results: {results}")
except Exception as e:
    print(f"Error analyzing results: {str(e)}")
    print(f"Raw results: {results}")

Found: del Guarlone - c.c. Gignoro, 19;23, Via del Gignoro, Varlungo, Quartiere 2, Firenze, Toscana, 50135, Italia
Importance score: 8.105668726630072e-05
Type: amenity/pharmacy
Address detail count: 13


In [33]:
results

[{'location': Location(del Guarlone - c.c. Gignoro, 19;23, Via del Gignoro, Varlungo, Quartiere 2, Firenze, Toscana, 50135, Italia, (43.7697108, 11.2983228, 0.0)),
  'quality': {'raw': {'place_id': 74326641,
    'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright',
    'osm_type': 'node',
    'osm_id': 6359357124,
    'lat': '43.7697108',
    'lon': '11.2983228',
    'class': 'amenity',
    'type': 'pharmacy',
    'place_rank': 30,
    'importance': 8.105668726630072e-05,
    'addresstype': 'amenity',
    'name': 'del Guarlone - c.c. Gignoro',
    'display_name': 'del Guarlone - c.c. Gignoro, 19;23, Via del Gignoro, Varlungo, Quartiere 2, Firenze, Toscana, 50135, Italia',
    'address': {'amenity': 'del Guarlone - c.c. Gignoro',
     'house_number': '19;23',
     'road': 'Via del Gignoro',
     'neighbourhood': 'Varlungo',
     'suburb': 'Quartiere 2',
     'city': 'Firenze',
     'county': 'Firenze',
     'ISO3166-2-lvl6': 'IT-FI',
     'state': 'Toscana'

In [ ]:
# Apply geocoding with tqdm progress bar
df["Latitude"], df["Longitude"] = zip(*[get_location_with_quality_indicators(addr, idx) for idx, addr in tqdm(enumerate(df["Address"]), total=len(df))])

# Display final result
display(df)

## Address Only

In [20]:
# Function to geocode an address with retry logic and live updates
def geocode_address(address, index):
    try:
        location = geolocator.geocode(address, timeout=10)
        if location:
            lat, lon = round(location.latitude, 6), round(location.longitude, 6)  # Round to 6 decimals
        else:
            lat, lon = None, None
    except GeocoderTimedOut:
        time.sleep(1)
        return geocode_address(address, index)  # Retry on timeout
    
    # Update widget display
    with output_widget:
        clear_output(wait=True)
        print(f"Geocoded {index + 1}/{len(df)}: {address} → {lat}, {lon}")
    
    return lat, lon

- <b>Importance score:</b> Nominatim provides an "importance" field which somewhat correlates with the prominence/significance of a place. Higher values generally mean more confidence.
- <b>Address detail count:</b> More complete address details often indicate a more accurate match.
- <b>Result type:</b> Check if the result type matches what you're looking for (e.g., building, street, city).

In [28]:
# Widget to display progress
output_widget = widgets.Output()
display(output_widget)

Output()

In [29]:
# Apply geocoding with tqdm progress bar
df["Latitude"], df["Longitude"] = zip(*[geocode_address(addr, idx) for idx, addr in tqdm(enumerate(df["Address"]), total=len(df))])

# Display final result
display(df)

  0%|          | 0/86 [00:00<?, ?it/s]

,ViaCompleta,ViaSenzaCivico,Civico,CAP,Comune,SiglaProvincia,Provincia,Address,Latitude,Longitude
CodiceIdentificativo,,,,,,,,,,
B06HKB1SUD,"VIA COMUNALE DELLE PUGLIE,171",VIA COMUNALE DELLE PUGLIE,171,80013,CASALNUOVO DI NAPOLI,NaN,NAPOLI,NaN,46.314475,11.048029
B0DE7B2QKW,"VIA GRAZIA,37/37",VIA GRAZIA,37/37,98057,MILAZZO,ME,MESSINA,"VIA GRAZIA, 37/37, MILAZZO, ME, MESSINA",38.208113,15.244937
B0GLSB2U60,"VIA VALLESELLA,24",VIA VALLESELLA,24,35046,BORGO VENETO,PD,PADOVA,"VIA VALLESELLA, 24, BORGO VENETO, PD, PADOVA",45.225303,11.524106
3381J58947,"VIA IRLANDA,11",VIA IRLANDA,11,50063,FIGLINE E INCISA VALDARNO,FI,FIRENZE,"VIA IRLANDA, 11, FIGLINE E INCISA VALDARNO, FI...",NaN,NaN
B08O8B2CJ3,VIA MACANNO SNC,VIA MACANNO,SNC,47923,RIMINI,RN,RIMINI,"VIA MACANNO, SNC, RIMINI, RN, RIMINI",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
F1MSZF3VWS,"TRAVERSA V DI VIA APPIA LATO NAPOLI,5",TRAVERSA V DI VIA APPIA LATO NAPOLI,NaN,4023,FORMIA,LT,LATINA,NaN,46.314475,11.048029
F0MV9F2OGA,"VIA NAZIONALE,12 T",VIA NAZIONALE,12/T,98138,MESSINA,ME,MESSINA,"VIA NAZIONALE, 12/T, MESSINA, ME, MESSINA",38.069965,15.487462
F1GJ9F3KZG,"VIA ELATE PENTESILEA,16",VIA ELATE PENTESILEA,16,24014,PIAZZA BREMBANA,BG,BERGAMO,"VIA ELATE PENTESILEA, 16, PIAZZA BREMBANA, BG,...",NaN,NaN


## Export Geocoded Values to Comma Delimited (csv) or/and Excel (xlsx)

In [30]:
df.to_csv('output.csv')

In [31]:
df.to_excel('output.xlsx')

# Convert to GIS Dataset (Point)

In [32]:
# Convert DataFrame to GeoDataFrame
df.dropna(subset=["Latitude", "Longitude"], inplace=True)  # Remove rows with missing coordinates
df["geometry"] = df.apply(lambda row: Point(row["Longitude"], row["Latitude"]), axis=1)  # Create Point geometries
## MODIFICA Geodetic CRS: RDN2008
gdf = gpd.GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")  # Set CRS to WGS 84

# Display final GeoDataFrame
display(gdf)

,ViaCompleta,ViaSenzaCivico,Civico,CAP,Comune,SiglaProvincia,Provincia,Address,Latitude,Longitude,geometry
CodiceIdentificativo,,,,,,,,,,,
B06HKB1SUD,"VIA COMUNALE DELLE PUGLIE,171",VIA COMUNALE DELLE PUGLIE,171,80013,CASALNUOVO DI NAPOLI,NaN,NAPOLI,NaN,46.314475,11.048029,POINT (11.04803 46.31448)
B0DE7B2QKW,"VIA GRAZIA,37/37",VIA GRAZIA,37/37,98057,MILAZZO,ME,MESSINA,"VIA GRAZIA, 37/37, MILAZZO, ME, MESSINA",38.208113,15.244937,POINT (15.24494 38.20811)
B0GLSB2U60,"VIA VALLESELLA,24",VIA VALLESELLA,24,35046,BORGO VENETO,PD,PADOVA,"VIA VALLESELLA, 24, BORGO VENETO, PD, PADOVA",45.225303,11.524106,POINT (11.52411 45.2253)
F1GIQF3KY2,VIA PIETRO COSSA 2934/B,VIA PIETRO COSSA,2934/B,10151,TORINO,TO,TORINO,"VIA PIETRO COSSA, 2934/B, TORINO, TO, TORINO",45.093999,7.632685,POINT (7.63268 45.094)
F0PSAF2RD1,VIA EVANGELISTA 36,VIA EVANGELISTA,36,81025,MARCIANISE,CE,CASERTA,"VIA EVANGELISTA, 36, MARCIANISE, CE, CASERTA",41.030905,14.312082,POINT (14.31208 41.0309)
F0LC1F2J25,"PIAZZA VITTORIA,16",PIAZZA VITTORIA,16,88900,CROTONE,KR,CROTONE,"PIAZZA VITTORIA, 16, CROTONE, KR, CROTONE",39.080722,17.127088,POINT (17.12709 39.08072)
F0IYLF233D,"VIA MARE,90",VIA MARE,90,73059,UGENTO,LE,LECCE,"VIA MARE, 90, UGENTO, LE, LECCE",39.921399,18.156464,POINT (18.15646 39.9214)
F0A2YF1B5G,VIA GIUSEPPE MAZZINI 2,VIA GIUSEPPE MAZZINI,2,46034,BORGO VIRGILIO,MN,MANTOVA,"VIA GIUSEPPE MAZZINI, 2, BORGO VIRGILIO, MN, M...",45.115341,10.785330,POINT (10.78533 45.11534)
F0TKSF2W7O,VIA DEL GIGNORO 19,VIA DEL GIGNORO,19,50135,FIRENZE,FI,FIRENZE,"VIA DEL GIGNORO, 19, FIRENZE, FI, FIRENZE",43.769711,11.298323,POINT (11.29832 43.76971)


In [33]:
gdf.explore()

# Reproject to EPSG:7794 RDN2008 / Italy zone (E-N)

In [38]:
# Reproject to EPSG:7794
gdf_7794 = gdf.to_crs(epsg=7794)

# Print CRS to verify
print(gdf_7794.crs)

EPSG:7794


## Export to shapefile

In [39]:
# Reset index to make "CodiceIdentificativo" a regular column
gdf_7794 = gdf_7794.reset_index()

In [40]:
# Rename columns to fit the 10-character limit
gdf_7794 = gdf_7794.rename(columns={
    "CodiceIdentificativo": "ID",
    "ViaCompleta": "indirizzo",
    "ViaSenzaCivico": "no_civico", # CONTROLLO
    "Civico": "civico",
    "Comune": "comune",
    "SiglaProvincia": "sigla",
    "Provincia": "provincia",
    "Address":"codifica",
    "Latitude": "Lat",
    "Longitude": "Lon"
})

In [41]:
gdf_7794.to_file('output.shp')